# METHOD 1

In [1]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import re
import pandas as pd
import numpy as np
import emoji
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [3]:
# Extract Time
def date_time(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    result = re.match(pattern, s)
    if result:
        return True
    return False

# Find Authors or Contacts
def find_author(s):
    s = s.split(":")
    if len(s)==2:
        return True
    else:
        return False

# Finding Messages
def getDatapoint(line):
    splitline = line.split(' - ')
    dateTime = splitline[0]
    date, time = dateTime.split(", ")
    message = " ".join(splitline[1:])
    if find_author(message):
        splitmessage = message.split(": ")
        author = splitmessage[0]
        message = " ".join(splitmessage[1:])
    else:
        author= None
    return date, time, author, message

In [50]:
# Load chat txt file into data list
data = []
conversation = 'WhatsApp Chat with NHCEIS Rishikesh.txt'
#conversation = 'WhatsApp Chat with Bipolar Builders.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        if date_time(line):
            if len(messageBuffer) > 0:
                data.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getDatapoint(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

In [51]:
df = pd.DataFrame(data, columns=["Date", 'Time', 'Author', 'Message'])
df['Date'] = pd.to_datetime(df['Date'])

In [52]:
Messages = df['Message']
print(Messages)

0                           Hey Rishikesh this is Mihir
1     I wanted to know on which chatbot website are ...
2             NHCEIS Rishikesh: I'll call you at 10:45?
3               Hello mihir, i have your number saved 🙂
4                                                Oh lol
5                                               Yeah np
6                            NHCEIS Rishikesh: Great :)
7                Gimme 20 minutes, I'll send you a link
8                                                  sure
9     NHCEIS Rishikesh: https://meet.google.com/fcp-...
10    NHCEIS Rishikesh: https://indiatourbangalore.d...
11                                d40eaa1132  team code
12                       The world is a depressed place
13                               War is the only answer
14                                 Life is at the worst
15                             I hate this unfair world
16                                           Life sucks
17                     There's no way out of thi

In [9]:
!pip install happytransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from happytransformer import HappyTextClassification

In [11]:
happt_tc = HappyTextClassification(model_type='DISTILBERT', model_name='distilbert-base-uncased-finetuned-sst-2-english', num_labels=2)

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [16]:
#for txt in Messages:
result = happt_tc.classify_text(Messages[0])

In [17]:
result.label

'POSITIVE'

In [18]:
result.score

0.9988100528717041

In [61]:
neg_msg = []
neg_count = 0
pos_msg = []
pos_count = 0
neutral_msg = []
neutral_count = 0

for txt in Messages:
  result = happt_tc.classify_text(txt)
  if result.label == 'POSITIVE' and result.score > 0.999:
    pos_count += 1
    pos_msg.append(list([txt, result.score, 'POS']))
  elif result.label == 'NEGATIVE' and result.score > 0.999:
    neg_count += 1
    neg_msg.append(list([txt, result.score, 'NEG']))
  else:
    neutral_count += 1
    neutral_msg.append(list([txt, result.score, result.label]))

print("Positive Count :", pos_count)
print("Negative Count :", neg_count)
print("Neutral Count :", neutral_count)

Positive Count : 1
Negative Count : 9
Neutral Count : 25


In [62]:
for txt in neg_msg:
  print(txt)

['The world is a depressed place', 0.9997156262397766, 'NEG']
['Life is at the worst', 0.9998090863227844, 'NEG']
['I hate this unfair world', 0.9991762042045593, 'NEG']
['Life sucks', 0.9996532201766968, 'NEG']
['I ate bad food', 0.9996172189712524, 'NEG']
['Food was stale as fuk', 0.9997881054878235, 'NEG']
['I feel hopeless, helpless and lost', 0.9993675351142883, 'NEG']
['My life sucks', 0.999495267868042, 'NEG']
['Ill beat the crap out of you', 0.9990241527557373, 'NEG']


In [63]:
for txt in pos_msg:
  print(txt)

['sure', 0.9997507929801941, 'POS']


In [64]:
for txt in neutral_msg:
  print(txt)

['Hey Rishikesh this is Mihir', 0.9988100528717041, 'POSITIVE']
['I wanted to know on which chatbot website are we working today', 0.996921718120575, 'NEGATIVE']
["NHCEIS Rishikesh: I'll call you at 10:45?", 0.9922631978988647, 'POSITIVE']
['Hello mihir, i have your number saved 🙂', 0.9745357632637024, 'NEGATIVE']
['Oh lol', 0.7139353156089783, 'POSITIVE']
['Yeah np', 0.9984303116798401, 'POSITIVE']
['NHCEIS Rishikesh: Great :)', 0.9980620741844177, 'POSITIVE']
["Gimme 20 minutes, I'll send you a link", 0.9409500360488892, 'POSITIVE']
['NHCEIS Rishikesh: https://meet.google.com/fcp-wkag-nid', 0.879991888999939, 'NEGATIVE']
['NHCEIS Rishikesh: https://indiatourbangalore.devfolio.co/overview', 0.8348322510719299, 'NEGATIVE']
['d40eaa1132  team code', 0.872350811958313, 'NEGATIVE']
['War is the only answer', 0.9987671375274658, 'NEGATIVE']
["There's no way out of this place", 0.9979189038276672, 'NEGATIVE']
['You deleted this message', 0.998936116695404, 'NEGATIVE']
['Fuck this shit', 0.9